In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

# Initialize the language model with a low temperature for more deterministic responses
llm = ChatOpenAI(temperature=0.1)

# Set up the memory buffer to keep track of the conversation history with a token limit
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,  # Limit on the number of tokens in memory
    memory_key="chat_history",  # Key used to store chat history
    return_messages=True,  # Return messages to be used in responses
)

# Define the chat prompt template, including system instructions and placeholders for chat history and user questions
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),  # System message setting the context
    MessagesPlaceholder(variable_name="chat_history"),  # Placeholder for conversation history
    ("human", "{question}")  # Placeholder for the human's question
])

# Define a function to load memory
def load_memory(input):
    # Print the input for debugging purposes
    print(input)
    # Return the chat history from memory
    return memory.load_memory_variables({})["chat_history"]

# Create the chain of operations to process the input, generate the prompt, and get a response from the model
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# Define a function to invoke the chain with a question and save the context to memory
def invoke_chain(question):
    # Invoke the chain with the given question and store the result
    result = chain.invoke({"question": question})
    # Save the input question and the model's response to memory
    memory.save_context({"input": question}, {"output": result.content})
    # Print the result for debugging purposes
    print(result)

# Example usage of the invoke_chain function with a specific question
invoke_chain("My name is Jonathan")


In [2]:
invoke_chain("my name is Jonathan")

{'question': 'My name is Jonathan'}
content='Hello Jonathan! How can I assist you today?'


In [3]:
invoke_chain("what is nmy name?")

{'question': 'My name is Jonathan'}
content='Got it, Jonathan! How can I assist you today?'
